In [1]:
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import v2

In [4]:
train_data = pd.read_csv("data/train.csv")
train_data

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera
...,...,...
18348,images/18348.jpg,aesculus_glabra
18349,images/18349.jpg,liquidambar_styraciflua
18350,images/18350.jpg,cedrus_libani
18351,images/18351.jpg,prunus_pensylvanica


In [ ]:
class LeaveDataset(Dataset):
    def __init__(self, x, y, train=True):
        self.x = x
        self.y = y
        self.train = train
        self.train_transpose = v2.Compose([
            v2.RandomHorizontalFlip(),
            v2.RandomVerticalFlip(),
            v2.RandomRotation(180, fill=(255, 255, 255)),
            v2.ColorJitter(0.5),
            v2.Resize((224, 224)),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, True),
            v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.test_transpose = v2.Compose([
            v2.Resize((224, 224)),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, True),
            v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.data_path = "data/images/"
        
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        x = Image.open(self.data_path + x)
        x = self.train_transpose(x) if self.train else self.test_transpose(x)
        return x, y
    
    def __len__(self):
        return len(self.x)